In [1]:
# import libraries 
#!pip install yfinance
import pandas as pd
import numpy as np
import yfinance  as yf
from openpyxl import load_workbook


In [2]:
#import excel sheet information as a dataframe
excel_path = (r"C:\Users\wills\OneDrive\uh\Coursea Analytics\Stock Positions.xlsx")
portfolio_df = pd.read_excel(excel_path,sheet_name = "Stock Positions")
portfolio_df

,Account Name,Symbol,Quantity,Current Stock Price,Original Paid Price,Current Holdings,Date Imported
0,INVESTMENT STOCKS,INTC,0.371,23.430000,19.96,-11.267470,2025-07-14
1,INVESTMENT STOCKS,TSLA,0.168,313.510010,39.68,12.989682,2025-07-14
2,INVESTMENT STOCKS,MA,0.137,550.179993,49.71,25.664659,2025-07-14
3,INVESTMENT STOCKS,BYND,0.484,3.510000,59.94,-58.241160,2025-07-14
4,INVESTMENT STOCKS,DAL,1.478,56.650002,59.99,23.738702,2025-07-14
5,INVESTMENT STOCKS,GOOGL,0.480,180.190002,67.95,18.541201,2025-07-14
6,INVESTMENT STOCKS,MSFT,0.329,503.320007,99.73,65.862282,2025-07-14
7,INVESTMENT STOCKS,AMZN,0.900,225.020004,149.05,53.468004,2025-07-14
8,INVESTMENT STOCKS,FCNTX,32.482,23.290001,591.99,164.515810,2025-07-14


In [3]:
#pull ticker symbols from dataset for yahoo finance historical data
ticker_column = "Symbol"  # or 'Ticker Symbol' depending on your export
tickers = portfolio_df[ticker_column].dropna().unique().tolist()

In [4]:
#Create that seperates mutual fund data and stock data
mutual_funds = []
stocks_etfs = []

for symbol in tickers:
    if symbol.endswith("X"):  # most mutual fund tickers end in X
        mutual_funds.append(symbol)
    else:
        stocks_etfs.append(symbol)

print("Stocks/ETFs:", stocks_etfs)
print("Mutual Funds:", mutual_funds)

Stocks/ETFs: ['INTC', 'TSLA', 'MA', 'BYND', 'DAL', 'GOOGL', 'MSFT', 'AMZN']
Mutual Funds: ['FCNTX']


In [ ]:
#Yahoo finance API to recieve stock data
stock_data = yf.download(stocks_etfs, start="2000-01-01", auto_adjust=True)
mutual_funds_data = yf.download(mutual_funds, start="2000-01-01", auto_adjust=True)

historical_stock_df = pd.concat([stock_data["Close"], mutual_funds_data["Close"]]) # Combines Stocks and Mutual Funds into One dataframe (excel sheet)
historical_stock_df.reset_index=True # Resets indexing to start over with new columns 
historical_stock_df.fillna("", inplace=True) # fills Null values with Blanks instead of Errors/random data
historical_stock_df = historical_stock_df.sort_values(by='Date', ascending=False) # Sorts the data from Descending data (starts at 2025 rather than 2000)
historical_stock_df


[*********************100%***********************]  8 of 8 completed
[*********************100%***********************]  1 of 1 completed
C:\Users\wills\AppData\Local\Temp\ipykernel_25688\772351990.py:7: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  historical_stock_df.fillna("", inplace=True)


Ticker,AMZN,BYND,DAL,GOOGL,INTC,MA,MSFT,TSLA,FCNTX
Date,,,,,,,,,
2025-07-14,225.690002,3.46,58.110001,181.559998,23.299999,553.02002,503.019989,316.899994,
2025-07-11,225.020004,3.51,56.650002,180.190002,23.43,550.179993,503.320007,313.51001,
2025-07-11,,,,,,,,,23.290001
2025-07-10,,,,,,,,,23.370001
2025-07-10,222.259995,3.59,56.779999,177.619995,23.82,563.52002,501.480011,309.869995,
...,...,...,...,...,...,...,...,...,...
2000-01-05,3.4875,,,,23.752048,,34.883846,,
2000-01-04,,,,,,,,,1.664327
2000-01-04,4.096875,,,,23.556782,,34.519859,,


In [6]:
#upload historical data to our existing excel sheet
workbook= load_workbook(excel_path)

with pd.ExcelWriter(excel_path, engine='openpyxl',mode='a', if_sheet_exists="replace") as writer:
        historical_stock_df.to_excel(writer, sheet_name="Historical Data", index="False")